In [1]:
import sys 
import os
from bh24_literature_mining.europepmc_api import EuropePMCClient
from bh24_literature_mining.utils import load_biotools_pub
from bh24_literature_mining.utils import load_biotools_from_zip
from bh24_literature_mining.utils import load_biotools_from_json
import numpy as np
import json
import pandas as pd
import requests
import igraph as ig


In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.7.0' # optional
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


SystemExit: 1

/home/ucloud/.cache/pypoetry/virtualenvs/bh24-literature-mining-9a6zG5on-py3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3040: UserWarning: Unknown failure executing module: <pip>
  warn('Unknown failure executing module: <%s>' % mod_name)


ModuleNotFoundError: No module named 'py4cytoscape'

In [ ]:
import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.7.2',
 'automationAPIVersion': '1.11.0',
 'py4cytoscapeVersion': '1.11.0'}

In [2]:
from bh24_literature_mining.europepmc_api import Article

tools = Article.read_cites_from_json("../var/biotools_cites.json")


print(f'Loaded {len(tools)} tools.')

Loaded 9453 tools.


### Remove preprints from articles in every tool

For published articles there can also be preprints included since DOI and other IDs are different from the final publication to the preprint.

In [3]:
for tool in tools:
    tool["articles"] = [
        article for article in tool["articles"] if article.pubType != "preprint"
    ]

print(f"Filtered out preprints. Now {len(tools)} tools.")

Filtered out preprints. Now 9453 tools.


### Get all publication IDs

In [4]:
all_ids = []
for tool in tools:
    for article in tool['articles']:
        all_ids.append(article.id)

all_ids = set(all_ids)

print(f"Total number of articles: {len(all_ids)}")

Total number of articles: 366828


### Create binary matrix for tools vs. articles

In [6]:
tool_names = [tool['name'] for tool in tools]
matrix = pd.DataFrame(0, index=tool_names, columns=sorted(set(all_ids)))
i = 0
for tool in tools:
    print(tool)
    print(f'{i}/{len(tools)}')
    i = i + 1
    for article in tool['articles']:   
        matrix.loc[ tool['name'], article.id] = 1

TypeError: string indices must be integers, not 'str'

In [10]:
matrix

,10811817,11125070,11178258,11574056,11597340,11869452,12019020,12171605,12877744,14611664,...,PPR7024,PPR7025,PPR7026,PPR7027,PPR7028,PPR7029,PPR7032,PPR7033,PPR7034,PPR7035
Seurat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bakta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gget,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SIBiLS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Variomes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MeDeCom,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MToolBox,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCBI Bookshelf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EXTRACT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
matrix.drop_duplicates(inplace=True)

In [25]:
matrix_filt = matrix.loc[:, matrix.sum(axis=0) > 1]

In [26]:
adj = np.dot(matrix.values, matrix.values.T)

In [27]:
adj_df = pd.DataFrame(adj, index=matrix.index, columns=matrix.index)

In [28]:
adj_df

,SubtiWiki,MetaPathways,iPHoP,MZmine,SLiM,Jalview,ITHANET,Fiji,BCFtools,SAMtools,...,HistoMIL,MATTE,Oli2go,scViewer,CavityPlus,3DVizSNP,CrossDome,PlasBin-flow,SPECK,IHM-DB
SubtiWiki,95,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MetaPathways,0,61,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
iPHoP,0,0,50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MZmine,0,0,0,94,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SLiM,0,0,0,0,86,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3DVizSNP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
CrossDome,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
PlasBin-flow,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
SPECK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,86,0


In [41]:
np.fill_diagonal(adj_df.values, 0)

In [45]:
# Get the values as np.array, it's more convenenient.
A = adj_df.values

# Create graph, A.astype(bool).tolist() or (A / A).tolist() can also be used.
g = ig.Graph.Adjacency((A > 0).tolist())

# Add edge weights and node labels.
g.es['weight'] = A[A.nonzero()]
g.vs['name'] = adj_df.columns  # or a.index/a.columns

In [47]:
# Check Cytoscape connection
p4c.cytoscape_ping()

# Convert igraph graph to Cytoscape and create the network
p4c.create_network_from_igraph(g, title="Comention Network", collection="Comention Network Collection")

# Set node and network visual properties
#p4c.set_node_color_bypass(pmids, "red")
p4c.set_node_shape_default("ELLIPSE")
p4c.set_node_width_default(30)
p4c.set_node_height_default(30)

# Compute shortest paths using igraph and display in console or store as needed


You are connected to Cytoscape!
Applying default style...
Applying preferred layout
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.


''